In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
from torch.autograd import *

import torchvision
import torchvision.transforms as transforms

import os
import argparse
import matplotlib.pyplot as plt
import numpy as np

from resnet import *

import pickle
from collections import OrderedDict

import torchattacks

import matplotlib.pyplot as plt

from torch.utils.data import DataLoader



/home/pratyush/miniconda3/envs/pytorch_3.9/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# import argparse
import os

# parser = argparse.ArgumentParser(description='PyTorch Cifar10 Training')
# parser.add_argument('--ngpu', type=int, default=1, help='0 = CPU.')
# parser.add_argument('--gpu_id', type=int, default=0,
#                     help='device range [0,ngpu-1]')


# args = parser.parse_args()

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
ngpu = 1
gpu_id = 1
if ngpu == 1:
    # make only devices indexed by #gpu_id visible
    os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_id)

```markdown
# Load Dataset
```

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
lr = 0.01

print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=100, shuffle=False, num_workers=2)

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

# Model
print('==> Building model..')
net2 = ResNet18()
net2 = net2.to(device)
if device == 'cuda':
    net2 = torch.nn.DataParallel(net2)
    cudnn.benchmark = True

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net2.parameters(), lr=lr,
                      momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)


==> Preparing data..


0it [00:00, ?it/s]

170500096it [00:09, 17881220.16it/s]                               


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
==> Building model..


```markdown
# Load Models
```

In [4]:
file_path_basic = "/home/pratyush/pratyushg/models/basicResnetModels/resnet18_cifar10_basicTraining.pth"
file_path_fgsm = "/home/pratyush/pratyushg/models/fgsmModels/resnet18_cifar10_fgsmTraining.pth"
file_path_pgd = "/home/pratyush/pratyushg/models/pgdModels/resnet18_githubPGD20_epoch200_better.pth"
file_path_chaosLoss = "/home/pratyush/pratyushg/models/chaosLossMinModels/resnet18_cifar10_chaos_regularized_pgd_torchattacks_lambdaChaos0.8_epoch200_lr0.01_iters20.pth"

In [5]:
# Load ResNet18 model
classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

print('==> Loading basic ResNet model..')
net_basic = ResNet18()
net_basic = net_basic.to(device)
if device == 'cuda':
    net_basic = torch.nn.DataParallel(net_basic)
    cudnn.benchmark = True

state_dict = torch.load(file_path_basic)

# Create a new state dictionary without the 'module.' prefix
new_state_dict = OrderedDict()
for k, v in state_dict.items():
    if k.startswith('module.'):
        name = k[7:]  # remove 'module.' prefix
    else:
        name = k
    new_state_dict[name] = v

net_basic.load_state_dict(state_dict)

#Load ResNet18 model with FGSM adversarial training
print('==> Loading FGSM-trained model..')
net_fgsm = ResNet18()
net_fgsm = net_fgsm.to(device)
if device == 'cuda':
    net_fgsm = torch.nn.DataParallel(net_fgsm)
    cudnn.benchmark = True
    
state_dict = torch.load(file_path_fgsm)

new_state_dict = OrderedDict()
for k, v in state_dict.items():
    if k.startswith('module.'):
        name = k[7:]  # remove 'module.' prefix
    else:
        name = k
    new_state_dict[name] = v

net_fgsm.load_state_dict(state_dict)

#Load ResNet18 model with PGD adversarial training
print('==> Loading PGD-trained model..')
net_pgd = ResNet18()
net_pgd = net_pgd.to(device)
if device == 'cuda':
    net_pgd = torch.nn.DataParallel(net_pgd)
    cudnn.benchmark = True

state_dict = torch.load(file_path_pgd)

new_state_dict = OrderedDict()
for k, v in state_dict.items():
    if k.startswith('module.'):
        name = k[7:]  # remove 'module.' prefix
    else:
        name = k
    new_state_dict[name] = v

net_pgd.load_state_dict(state_dict)

#Load ResNet18 model with Chaos Loss minimization training
print('==> Loading ChaosLoss Minimization model..')
net_chaosLoss = ResNet18()
net_chaosLoss = net_chaosLoss.to(device)
if device == 'cuda':
    net_chaosLoss = torch.nn.DataParallel(net_chaosLoss)
    cudnn.benchmark = True

state_dict = torch.load(file_path_chaosLoss)

new_state_dict = OrderedDict()
for k, v in state_dict.items():
    if k.startswith('resnet18.'):
        name = k.replace('resnet18', 'module')
    else:
        name = k
    new_state_dict[name] = v

net_chaosLoss.load_state_dict(new_state_dict)


==> Loading basic ResNet model..
==> Loading FGSM-trained model..
==> Loading PGD-trained model..
==> Loading ChaosLoss Minimization model..


<All keys matched successfully>

```markdown
# Normal Dataset Evaluation
```

In [6]:
def test(epoch, net):

    '''
    This function evaluate net on test dataset
    '''

    global acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    acc = 100 * correct / total
    return test_loss/len(testloader)

In [7]:
test_loss = test(1, net_chaosLoss)
print('net_chaos on default dataset accuracy: %d %%' % (acc))

net_chaos on default dataset accuracy: 87 %


```markdown
# FGSM Attack Evaluation
```

In [8]:
# Copy model

print('==> Loading basic ResNet model..')
net_basic_copy = ResNet18()
net_basic_copy = net_basic_copy.to(device)
if device == 'cuda':
    net_basic_copy = torch.nn.DataParallel(net_basic_copy)
    cudnn.benchmark = True

state_dict = torch.load(file_path_basic)

# Create a new state dictionary without the 'module.' prefix
new_state_dict = OrderedDict()
for k, v in state_dict.items():
    if k.startswith('module.'):
        name = k[7:]  # remove 'module.' prefix
    else:
        name = k
    new_state_dict[name] = v

net_basic_copy.load_state_dict(state_dict)

==> Loading basic ResNet model..


<All keys matched successfully>

In [9]:
def FGSM(net, x, y, eps):
        '''
        inputs:
            net: the network through which we pass the inputs
            x: the original example which we aim to perturb to make an adversarial example
            y: the true label of x
            eps: perturbation budget

        outputs:
            x_adv : the adversarial example constructed from x
            h_adv: output of the last softmax layer when applying net on x_adv 
            y_adv: predicted label for x_adv
            pert: perturbation applied to x (x_adv - x)
        '''

        x_ = Variable(x.data, requires_grad=True)
        h_ = net(x_)
        criterion= torch.nn.CrossEntropyLoss()
        cost = criterion(h_, y)
        net.zero_grad()
        cost.backward()

        #perturbation
        pert= eps*x_.grad.detach().sign()
        
        x_adv = x_ + pert

        h_adv = net(x_adv)
        _,y_adv=torch.max(h_adv.data,1)
        return x_adv, h_adv, y_adv, pert


In [10]:
def test_fgsm(net, net_adv, eps):
    accuracy=0
    net.train()
    net_adv.eval()
    test_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(testloader):
        inputs, targets = inputs.to(device), targets.to(device)

        x_adv, h_adv, y_adv, pert = FGSM (net, inputs, targets, eps)
            
        outputs = net_adv(x_adv)
        loss = criterion(outputs, targets)

        test_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
    accuracy = 100 * correct / total
    return accuracy

In [11]:
accuracy=test_fgsm(net_basic_copy, net_chaosLoss, 8/255)
print("accuracy of net_chaosLoss on FGSM-attacked dataset:", accuracy)

accuracy of net_chaosLoss on FGSM-attacked dataset: 86.82


In [12]:
# Define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the pretrained ResNet18 model
model = net_chaosLoss
model.eval()

# Define the FGSM attack
epsilon = 8/255  # Perturbation

fgsm = torchattacks.FGSM(model, eps=epsilon)

# Function to test the model on adversarial examples
def test_model_on_adversarial(loader, model, attack):
    correct = 0
    total = 0
    model.eval()
    
    for data in loader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)

        # Generate adversarial examples
        adv_images = attack(images, labels)

        # Forward pass the adversarial examples through the model
        outputs = model(adv_images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Accuracy of the model on FGSM examples: {100 * correct / total:.2f}%')

# Test the model on the adversarially-attacked CIFAR-10 test set
test_model_on_adversarial(testloader, model, fgsm)


Accuracy of the model on FGSM examples: 66.04%


```markdown
# PGD Attack Evaluation
```

In [13]:
# Load the pretrained ResNet18 model
model = net_pgd
model.eval()

# Define the PGD attack
epsilon = 8/255  # Perturbation
alpha = 2/255    # Step size
steps = 20        # Number of iterations

pgd = torchattacks.PGD(model, eps=epsilon, alpha=alpha, steps=steps)

# Function to test the model on adversarial examples
def test_model_on_adversarial(loader, model, attack):
    correct = 0
    total = 0
    model.eval()
    
    for data in loader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)

        # Generate adversarial examples
        adv_images = attack(images, labels)

        # Forward pass the adversarial examples through the model
        outputs = model(adv_images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Accuracy of the model on PGD examples: {100 * correct / total:.2f}%')

# Test the model on the adversarially-attacked CIFAR-10 test set
test_model_on_adversarial(testloader, model, pgd)


Accuracy of the model on PGD examples: 46.07%


```markdown
# Susceptibility Comparison
```

In [14]:
# Define a function to calculate the susceptibility ratio
def susceptibility_ratio(net, x, delta_x_adv):
    x = x.to(device)
    delta_x_adv = delta_x_adv.to(device)
    
    # Ensure the model is in evaluation mode
    net.eval()
    
    # Calculate h(θ; x_i)
    output_x = net(x)
    
    # Calculate h(θ; x_i + δx_adv)
    output_x_adv = net(x + delta_x_adv)
    
    # Calculate the susceptibility ratio
    num = torch.norm(output_x - output_x_adv, p=2)
    denom = torch.norm(delta_x_adv, p=2)
    
    susceptibility = torch.exp(num / denom)
    return susceptibility.item()

# Get a batch of training data
dataiter = iter(trainloader)
images, labels = dataiter.next()

# Define a small adversarial perturbation
epsilon = 0.01
delta_x_adv = epsilon * torch.sign(torch.randn_like(images)) #FGSM Attack

# Calculate the susceptibility ratio for the batch
susceptibility_basic_model = susceptibility_ratio(net_basic, images, delta_x_adv)
susceptibility_fgsm_model = susceptibility_ratio(net_fgsm, images, delta_x_adv)
susceptibility_pgd_model = susceptibility_ratio(net_pgd, images, delta_x_adv)
susceptibility_chaosLoss_model = susceptibility_ratio(net_chaosLoss, images, delta_x_adv)
print(f'Susceptibility Ratio of Basic Model: {susceptibility_basic_model}')
print(f'Susceptibility Ratio of Adversarially Trained (FGSM) Model: {susceptibility_fgsm_model}')
print(f'Susceptibility Ratio of Adversarially Trained (PGD) Model: {susceptibility_pgd_model}')
print(f'Susceptibility Ratio of Chaos-loss trained Model: {susceptibility_chaosLoss_model}')

Susceptibility Ratio of Basic Model: 1.490615725517273
Susceptibility Ratio of Adversarially Trained (FGSM) Model: 1.060843825340271
Susceptibility Ratio of Adversarially Trained (PGD) Model: 1.0887888669967651
Susceptibility Ratio of Chaos-loss trained Model: 1.0358573198318481


In [20]:
layers_to_capture = [
    'conv1', 'bn1',
    'layer1.0.conv1', 'layer1.0.bn1', 'layer1.0.conv2', 'layer1.0.bn2',
    'layer1.1.conv1', 'layer1.1.bn1', 'layer1.1.conv2', 'layer1.1.bn2',
    'layer2.0.conv1', 'layer2.0.bn1', 'layer2.0.conv2', 'layer2.0.bn2', 'layer2.0.shortcut.0', 'layer2.0.shortcut.1',
    'layer2.1.conv1', 'layer2.1.bn1', 'layer2.1.conv2', 'layer2.1.bn2',
    'layer3.0.conv1', 'layer3.0.bn1', 'layer3.0.conv2', 'layer3.0.bn2', 'layer3.0.shortcut.0', 'layer3.0.shortcut.1',
    'layer3.1.conv1', 'layer3.1.bn1', 'layer3.1.conv2', 'layer3.1.bn2',
    'layer4.0.conv1', 'layer4.0.bn1', 'layer4.0.conv2', 'layer4.0.bn2', 'layer4.0.shortcut.0', 'layer4.0.shortcut.1',
    'layer4.1.conv1', 'layer4.1.bn1', 'layer4.1.conv2', 'layer4.1.bn2',
    'linear'
]

# Define a function to register hooks and capture intermediate outputs
def register_hooks(model):
    activations = {}
    
    def get_activation(name):
        def hook(model, input, output):
            activations[name] = output
        return hook
    
    for name, layer in model.named_modules():
        layer.register_forward_hook(get_activation(name))
    
    return activations

# Register hooks for all layers
activations_basic = register_hooks(net_basic)
activations_fgsm = register_hooks(net_fgsm)
activations_pgd = register_hooks(net_pgd)
activations_chaosLoss = register_hooks(net_chaosLoss)

# Define a function to calculate the susceptibility ratio
def susceptibility_ratio(net, x, labels, activations):
    
    x = x.to(device)
    pgd = torchattacks.PGD(net, eps=0.03, alpha=0.01, steps=10)
    delta_x_adv = pgd(x, labels) - x
    delta_x_adv = delta_x_adv.to(device)
    # print(torch.norm(delta_x_adv, p=2))
    
    # Ensure the model is in evaluation mode
    net.eval()
    
    # Clear previous activations
    activations.clear()
    
    # Forward pass for original input
    _ = net(x)
    output_x_layers = activations.copy()
    
    # Clear previous activations
    activations.clear()
    
    # Forward pass for perturbed input
    _ = net(x + delta_x_adv)
    output_x_adv_layers = activations.copy()
    
    susceptibilities = {}
    for layer in output_x_layers.keys():
        output_x = output_x_layers[layer]
        output_x_adv = output_x_adv_layers[layer]
        
        # Calculate the susceptibility ratio for each layer
        num = torch.norm(output_x - output_x_adv, p=2)
        denom = torch.norm(delta_x_adv, p=2)
        susceptibility = num / denom
        # print(num.item(), denom.item(), susceptibility.item())
        susceptibilities[layer] = susceptibility.item()
    
    return susceptibilities

# Get a batch of training data
dataiter = iter(trainloader)
images, labels = dataiter.next()

# Define a small adversarial perturbation
# epsilon = 0.01
# delta_x_adv = epsilon * torch.sign(torch.randn_like(images))


    
# Calculate the susceptibility ratio for the batch
susceptibilities_basic = susceptibility_ratio(net_basic, images, labels, activations_basic)
# susceptibilities_fgsm = susceptibility_ratio(net_fgsm, images, labels, activations_fgsm)
susceptibilities_pgd = susceptibility_ratio(net_pgd, images, labels, activations_pgd)
susceptibilities_chaosLoss = susceptibility_ratio(net_chaosLoss, images, labels, activations_chaosLoss)
if 'conv1' in susceptibilities_basic:
    sus_basic = susceptibilities_basic['conv1']
    print("hello")
else:
    sus_basic = 0.0
    print("hello2")
# Print the susceptibility ratios in a table format
print("Susceptibility Ratio Comparison when the perbutation is generated by iterative attacks:")
print("{:<25} {:<15} {:<15} {:<15} ".format('Layer', 'Basic Model', 'PGD Model', 'ChaosMin Model (λ=0.8)'))
for layer in layers_to_capture:
    sus_basic = susceptibilities_basic[layer]
    # sus_fgsm = susceptibilities_fgsm[layer]
    sus_pgd = susceptibilities_pgd[layer]
    sus_chaos = susceptibilities_chaosLoss[layer]
    print("{:<25} {:<15.6f} {:<15.6f} {:<15.6f}".format(layer, sus_basic, sus_pgd, sus_chaos))

Susceptibility Ratio Comparison when the perbutation is generated by iterative attacks:
Layer                     Basic Model     PGD Model       ChaosMin Model (λ=0.8) 


KeyError: 'conv1'

```markdown
# Histogram for Weight Distribution
```

In [ ]:
# Load your models
model1 = net_pgd  # Assuming this is defined somewhere in your code
model2 = net_chaosLoss

# Function to get convolutional layers
def get_conv_layers(model):
    conv_layers = []
    for name, module in model.named_modules():
        if isinstance(module, torch.nn.Conv2d):
            conv_layers.append((name, module))
    return conv_layers

# Function to plot histograms of weights for convolutional layers of two models
def plot_conv_weight_histograms(model1, model2):
    conv_layers1 = get_conv_layers(model1)
    conv_layers2 = get_conv_layers(model2)
    
    num_layers = len(conv_layers1)
    fig, axes = plt.subplots(num_layers, 2, figsize=(12, num_layers * 1.5))

    for i, ((name1, layer1), (name2, layer2)) in enumerate(zip(conv_layers1, conv_layers2)):
        weight1 = layer1.weight.data.cpu().numpy().flatten()
        weight2 = layer2.weight.data.cpu().numpy().flatten()

        axes[i, 0].hist(weight1, bins=50)
        axes[i, 0].set_title(f'Model 1: {name1}')
        axes[i, 0].set_xlabel('Weight value')
        axes[i, 0].set_ylabel('Frequency')

        axes[i, 1].hist(weight2, bins=50)
        axes[i, 1].set_title(f'Model 2: {name2}')
        axes[i, 1].set_xlabel('Weight value')
        axes[i, 1].set_ylabel('Frequency')

    plt.tight_layout()
    plt.show()

# Plot histograms of the weights for convolutional layers of both models
plot_conv_weight_histograms(model1, model2)
